In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.rcParams['axes.grid'] = False
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

In [4]:
from helper_functions_volume import *

In [7]:
import eikon as ek
import plotly.graph_objects as go
ek.set_app_key('9a249e0411184cf49e553b61a6e76c52d295ec17')

In [9]:
ticker = '.NSEI'

if not os.path.exists(f'{ticker}_MRStdev/SelectedStrategies'):
    os.makedirs(f'{ticker}_MRStdev/SelectedStrategies')

print(f"Processing {ticker}")
temp_og, temp_og1 = get_data(".NSEI")
temp_og = prepare_volume_features_from_cache(temp_og, True)
temp_og = add_volume_stdevs(temp_og)
dates_all_ss = pd.date_range(start=str(temp_og.iloc[1]['Date'] + timedelta(days=503))[:10], end="2024-06-15", freq=f'3M')
dates = valid_dates(dates_all_ss)

Processing .NSEI


In [26]:
with open(f'NSEI_Volume_MeanRevertingStdev_Backtest_Azure_temp_og.pkl',
          'wb') as file:
    pickle.dump(temp_og, file)


In [ ]:
results = BFO_vol_mr_stdev_strategy(temp_og)

In [ ]:
res_test2 = select_all_strategies_mr_stdev(24,dates, temp_og, ticker,save=True)
res_test4 = select_all_strategies_mr_stdev(48,dates, temp_og, ticker,save=True)
res_test8 = select_all_strategies_mr_stdev(96,dates, temp_og, ticker,save=True)

In [ ]:
t = time.time()
selected_strat = 1
strat = vol_mean_reverting_stdev_strategy(temp_og, levelb=results.iloc[selected_strat]["levelb"], lookbackb=results.iloc[selected_strat]["lookbackb"], band_buy=results.iloc[selected_strat]["band_buy"], xtp=results.iloc[selected_strat]["xtp"],
                                          ktp = results.iloc[selected_strat]["ktp"],leveltp=results.iloc[selected_strat]["leveltp"],lookbacktp=results.iloc[selected_strat]["lookbacktp"], xsl=results.iloc[selected_strat]["xsl"],
                                          ksl = results.iloc[selected_strat]["ksl"],levelsl=results.iloc[selected_strat]["levelsl"],lookbacksl=results.iloc[selected_strat]["lookbacksl"])
strat.generate_signals()
strat.signal_performance(10000,6)
elapsed = time.time() - t
print(elapsed)

strat.generate_signals(charts=True)
strat.plot_performance()
strat.daywise_performance['avg_sortino_of_trades']